In [1]:
%load_ext autoreload
%autoreload 2

# Analytics
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier

# Plotting
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")

# Utilities
import os
import joblib
from functools import reduce
from utils import *

RANDOM_STATE = 55 ## We will pass it to every sklearn call so we ensure reproducibility

In [5]:
data_path = os.path.dirname(os.getcwd()) + '/data'
data_prefix = '/broad_assets_'
rets = pd.read_excel(data_path + data_prefix + 'weekly_rets.xlsx', index_col=0, parse_dates=True)
rets = rets.shift(-1)
lstm_selections = pd.read_excel(data_path + data_prefix + 'lstm_test_data.xlsx', sheet_name=0, index_col=0, parse_dates=True)
lstm_selections

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0,0,1,1,0,0,0,0,0,1,1
2020-06-01,0,0,1,1,0,0,0,0,0,1,1
2020-06-08,0,0,1,1,0,0,0,0,0,1,1
2020-06-15,0,0,1,1,0,0,0,0,1,1,1
2020-06-22,0,0,1,0,0,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,1,0,1,1,1,1,0,1,0,0,1
2024-05-13,1,0,1,1,1,1,0,1,0,1,0
2024-05-20,0,1,1,1,1,1,0,1,0,1,0


In [7]:
lstm_counts = lstm_selections.sum(axis=1)
lstm_selections = lstm_selections.div(lstm_counts, axis=0)
lstm_selections

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0.0000,0.0000,0.2500,0.2500,0.0000,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500
2020-06-01,0.0000,0.0000,0.2500,0.2500,0.0000,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500
2020-06-08,0.0000,0.0000,0.2500,0.2500,0.0000,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500
2020-06-15,0.0000,0.0000,0.2000,0.2000,0.0000,0.0000,0.0000,0.0000,0.2000,0.2000,0.2000
2020-06-22,0.0000,0.0000,0.2500,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500,0.2500,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,0.1429,0.0000,0.1429,0.1429,0.1429,0.1429,0.0000,0.1429,0.0000,0.0000,0.1429
2024-05-13,0.1429,0.0000,0.1429,0.1429,0.1429,0.1429,0.0000,0.1429,0.0000,0.1429,0.0000
2024-05-20,0.0000,0.1429,0.1429,0.1429,0.1429,0.1429,0.0000,0.1429,0.0000,0.1429,0.0000


In [8]:
eval_rets = rets.loc[lstm_selections.index]
eval_rets

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0.0693,0.0396,-0.0333,0.0020,0.0729,0.1504,0.1502,0.1211,-0.0414,0.0052,0.0444
2020-06-01,0.1186,0.0639,-0.0713,-0.0230,0.1934,0.1900,0.1436,0.1353,-0.0250,-0.0474,0.0732
2020-06-08,-0.1029,-0.0278,0.0801,0.0200,-0.1535,-0.1175,-0.1059,-0.0825,0.0072,0.0312,-0.0336
2020-06-15,0.0337,0.0734,-0.0067,-0.0004,0.0189,-0.0384,0.0361,0.0580,0.0067,0.0340,0.0555
2020-06-22,-0.0423,-0.0344,0.0282,0.0088,-0.0193,-0.0434,-0.0248,-0.0148,0.0108,0.0209,-0.0333
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,0.0155,0.0112,0.0066,-0.0013,0.0006,0.0430,0.0507,0.0134,0.0035,0.0091,0.0102
2024-05-13,0.0336,0.0523,0.0152,0.0052,0.0391,0.0296,0.0265,0.0369,-0.0122,0.0759,0.0400
2024-05-20,-0.0021,0.0135,-0.0019,-0.0072,-0.0318,-0.0609,-0.0117,-0.0287,0.0034,-0.0756,-0.0354


In [9]:
# Preliminary portfolio summaries to ensure consistency
lstm_rets = (eval_rets * lstm_selections).sum(axis=1).to_frame('LSTM Portfolio Returns')
lstm_rets.describe()

,LSTM Portfolio Returns
count,211.0000
mean,0.0018
std,0.0288
min,-0.1011
25%,-0.0105
50%,0.0028
75%,0.0200
max,0.0895


# Benchmarks - Equal Weight Portfolio and Simple Momentum

We provide a portfolio that is equally weighted across all available assets in the investible universe to serve as a benchmark to the classifier model strategies' returns.

## Equal-Weights Portfolio

In [10]:
equal_rets = (eval_rets * 1/len(eval_rets.columns)).sum(axis=1).to_frame('Equal Weight Portfolio Return')
equal_rets.describe()

,Equal Weight Portfolio Return
count,211.0000
mean,0.0030
std,0.0294
min,-0.1040
25%,-0.0129
50%,0.0031
75%,0.0202
max,0.0770


For the simple momentum benchmarks, we take a few naive approaches:

1. We look at the prior holding period's return of a given asset
1. We look at the trailing 1-month return of a given asset
1. We look at the trailing 3-month return of a given asset
1. We look at the trailing 1-year return of a given asset

If that return was positive, we select it for investment in the following period. Otherwise, it is excluded from the portfolio. With these selections in hand, we equally-weight the investments.

**Note:** At this time, we are not making a weighted decision with respect to results from each momentum metric.

In [12]:
lag1_selections = pd.DataFrame(np.where(rets.shift(1) > 0, 1, 0), columns=rets.columns, index=rets.index)
lag1_selections = lag1_selections.loc[lstm_selections.index]

lag4 = rets.rolling(window=4).apply(lambda x: np.prod(1 + x) - 1, raw=False)
lag4_selections = pd.DataFrame(np.where(lag4.shift(1) > 0, 1, 0), columns=rets.columns, index=rets.index)
lag4_selections = lag4_selections.loc[lstm_selections.index]

lag12 = rets.rolling(window=12).apply(lambda x: np.prod(1 + x) - 1, raw=False)
lag12_selections = pd.DataFrame(np.where(lag12.shift(1) > 0, 1, 0), columns=rets.columns, index=rets.index)
lag12_selections = lag12_selections.loc[lstm_selections.index]

lag52 = rets.rolling(window=52).apply(lambda x: np.prod(1 + x) - 1, raw=False)
lag52_selections = pd.DataFrame(np.where(lag52.shift(1) > 0, 1, 0), columns=rets.columns, index=rets.index)
lag52_selections = lag52_selections.loc[lstm_selections.index]

display(lag1_selections)
display(lag4_selections)
display(lag12_selections)
display(lag52_selections)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,1,1,1,1,1,1,0,0,1,1,0
2020-06-01,1,1,0,1,1,1,1,1,0,1,1
2020-06-08,1,1,0,0,1,1,1,1,0,0,1
2020-06-15,0,0,1,1,0,0,0,0,1,1,0
2020-06-22,1,1,0,0,1,0,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,1,1,1,1,1,1,1,1,0,0,0
2024-05-13,1,1,1,0,1,1,1,1,1,1,1
2024-05-20,1,1,1,1,1,1,1,1,0,1,1


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,1,1,0,1,1,0,1,1,0,1,1
2020-06-01,1,1,0,1,1,1,1,1,0,1,1
2020-06-08,1,1,0,0,1,1,1,1,0,0,1
2020-06-15,1,1,1,1,1,1,1,1,0,0,1
2020-06-22,1,1,0,0,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,0,0,0,0,0,0,1,1,1,0,0
2024-05-13,1,1,1,1,1,1,1,1,0,0,0
2024-05-20,1,1,1,1,1,1,1,1,0,1,1


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0,1,1,1,0,0,0,0,1,1,0
2020-06-01,1,1,0,0,1,0,1,0,1,1,0
2020-06-08,1,1,0,0,1,1,1,1,0,1,1
2020-06-15,1,1,0,1,1,1,1,1,0,1,1
2020-06-22,1,1,0,0,1,1,1,1,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,1,1,0,0,0,0,1,1,1,1,1
2024-05-13,1,1,0,0,1,0,1,1,1,1,1
2024-05-20,1,1,0,0,1,1,1,1,1,1,1


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,1,1,1,1,0,0,0,0,1,1,0
2020-06-01,1,1,1,1,0,0,0,0,1,1,0
2020-06-08,1,1,1,1,0,0,0,0,1,1,0
2020-06-15,1,1,1,1,0,0,0,0,1,1,0
2020-06-22,1,1,1,1,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,1,1,0,0,1,0,1,1,1,1,1
2024-05-13,1,1,0,0,1,1,1,1,1,1,1
2024-05-20,1,1,0,0,1,1,1,1,1,1,1


In [13]:
lag1_counts = lag1_selections.sum(axis=1)
lag1_selections = lag1_selections.div(lag1_counts, axis=0)

lag4_counts = lag4_selections.sum(axis=1)
lag4_selections = lag4_selections.div(lag4_counts, axis=0)

lag12_counts = lag12_selections.sum(axis=1)
lag12_selections = lag12_selections.div(lag12_counts, axis=0)

lag52_counts = lag52_selections.sum(axis=1)
lag52_selections = lag52_selections.div(lag52_counts, axis=0)

display(lag1_selections)
display(lag4_selections)
display(lag12_selections)
display(lag52_selections)

,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.0000,0.0000,0.1250,0.1250,0.0000
2020-06-01,0.1111,0.1111,0.0000,0.1111,0.1111,0.1111,0.1111,0.1111,0.0000,0.1111,0.1111
2020-06-08,0.1429,0.1429,0.0000,0.0000,0.1429,0.1429,0.1429,0.1429,0.0000,0.0000,0.1429
2020-06-15,0.0000,0.0000,0.2500,0.2500,0.0000,0.0000,0.0000,0.0000,0.2500,0.2500,0.0000
2020-06-22,0.1250,0.1250,0.0000,0.0000,0.1250,0.0000,0.1250,0.1250,0.1250,0.1250,0.1250
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.0000,0.0000,0.0000
2024-05-13,0.1000,0.1000,0.1000,0.0000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000
2024-05-20,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0000,0.1000,0.1000


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0.1250,0.1250,0.0000,0.1250,0.1250,0.0000,0.1250,0.1250,0.0000,0.1250,0.1250
2020-06-01,0.1111,0.1111,0.0000,0.1111,0.1111,0.1111,0.1111,0.1111,0.0000,0.1111,0.1111
2020-06-08,0.1429,0.1429,0.0000,0.0000,0.1429,0.1429,0.1429,0.1429,0.0000,0.0000,0.1429
2020-06-15,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.0000,0.0000,0.1111
2020-06-22,0.1250,0.1250,0.0000,0.0000,0.1250,0.1250,0.1250,0.1250,0.0000,0.1250,0.1250
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.3333,0.3333,0.3333,0.0000,0.0000
2024-05-13,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.1250,0.0000,0.0000,0.0000
2024-05-20,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.1000,0.0000,0.1000,0.1000


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0.0000,0.2000,0.2000,0.2000,0.0000,0.0000,0.0000,0.0000,0.2000,0.2000,0.0000
2020-06-01,0.1667,0.1667,0.0000,0.0000,0.1667,0.0000,0.1667,0.0000,0.1667,0.1667,0.0000
2020-06-08,0.1250,0.1250,0.0000,0.0000,0.1250,0.1250,0.1250,0.1250,0.0000,0.1250,0.1250
2020-06-15,0.1111,0.1111,0.0000,0.1111,0.1111,0.1111,0.1111,0.1111,0.0000,0.1111,0.1111
2020-06-22,0.1250,0.1250,0.0000,0.0000,0.1250,0.1250,0.1250,0.1250,0.0000,0.1250,0.1250
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,0.1429,0.1429,0.0000,0.0000,0.0000,0.0000,0.1429,0.1429,0.1429,0.1429,0.1429
2024-05-13,0.1250,0.1250,0.0000,0.0000,0.1250,0.0000,0.1250,0.1250,0.1250,0.1250,0.1250
2024-05-20,0.1111,0.1111,0.0000,0.0000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111


,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,Asset 10,Asset 11
Date,,,,,,,,,,,
2020-05-22,0.1667,0.1667,0.1667,0.1667,0.0000,0.0000,0.0000,0.0000,0.1667,0.1667,0.0000
2020-06-01,0.1667,0.1667,0.1667,0.1667,0.0000,0.0000,0.0000,0.0000,0.1667,0.1667,0.0000
2020-06-08,0.1667,0.1667,0.1667,0.1667,0.0000,0.0000,0.0000,0.0000,0.1667,0.1667,0.0000
2020-06-15,0.1667,0.1667,0.1667,0.1667,0.0000,0.0000,0.0000,0.0000,0.1667,0.1667,0.0000
2020-06-22,0.1667,0.1667,0.1667,0.1667,0.0000,0.0000,0.0000,0.0000,0.1667,0.1667,0.0000
...,...,...,...,...,...,...,...,...,...,...,...
2024-05-06,0.1250,0.1250,0.0000,0.0000,0.1250,0.0000,0.1250,0.1250,0.1250,0.1250,0.1250
2024-05-13,0.1111,0.1111,0.0000,0.0000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111
2024-05-20,0.1111,0.1111,0.0000,0.0000,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111,0.1111


In [14]:
# Preliminary portfolio summaries to ensure consistency
lag1_rets = (eval_rets * lag1_selections).sum(axis=1).to_frame('1-Period Momentum Portfolio')
lag4_rets = (eval_rets * lag4_selections).sum(axis=1).to_frame('1-Month Momentum Portfolio')
lag12_rets = (eval_rets * lag12_selections).sum(axis=1).to_frame('3-Month Momentum Portfolio')
lag52_rets = (eval_rets * lag52_selections).sum(axis=1).to_frame('1-Year Momentum Portfolio')

benchmark_rets = pd.concat([equal_rets, lag1_rets, lag4_rets, lag12_rets, lag52_rets], axis=1)
benchmark_rets.describe()

,Equal Weight Portfolio Return,1-Period Momentum Portfolio,1-Month Momentum Portfolio,3-Month Momentum Portfolio,1-Year Momentum Portfolio
count,211.0000,211.0000,211.0000,211.0000,211.0000
mean,0.0030,0.0019,0.0034,0.0028,0.0022
std,0.0294,0.0297,0.0304,0.0322,0.0286
min,-0.1040,-0.0945,-0.1298,-0.0873,-0.0703
25%,-0.0129,-0.0168,-0.0119,-0.0183,-0.0177
50%,0.0031,0.0037,0.0037,0.0026,0.0036
75%,0.0202,0.0193,0.0200,0.0220,0.0197
max,0.0770,0.1077,0.0978,0.1616,0.0853


# Performance Evaluation

In [15]:
all_rets = pd.concat([benchmark_rets, lstm_rets], axis=1)
calc_summary_statistics(all_rets, annual_factor=52, correlations=False, keep_columns=['Annualized Mean',
                                                                                      'Annualized Vol',
                                                                                      'Min',
                                                                                      'Max',
                                                                                      'Skewness',
                                                                                      'Excess Kurtosis',
                                                                                      'Historical VaR',
                                                                                      'Historical CVaR',
                                                                                      'Max Drawdown',
                                                                                      'Peak',
                                                                                      'Bottom',
                                                                                      'Recover',
                                                                                      'Duration (days)']).T

Assuming excess returns were provided to calculate Sharpe. If returns were provided (steady of excess returns), the column "Sharpe" is actually "Mean/Volatility"


,Equal Weight Portfolio Return,1-Period Momentum Portfolio,1-Month Momentum Portfolio,3-Month Momentum Portfolio,1-Year Momentum Portfolio,LSTM Portfolio Returns
Annualized Mean,0.1577,0.0990,0.1744,0.1455,0.1146,0.0953
Annualized Vol,0.2117,0.2142,0.2190,0.2318,0.2063,0.2075
Min,-0.1040,-0.0945,-0.1298,-0.0873,-0.0703,-0.1011
Max,0.0770,0.1077,0.0978,0.1616,0.0853,0.0895
Skewness,-0.3049,-0.1255,-0.3267,0.4273,0.0630,-0.4329
Excess Kurtosis,1.0219,1.2685,2.3335,2.3746,0.0750,1.2165
Historical VaR (5.00%),-0.0422,-0.0516,-0.0430,-0.0476,-0.0471,-0.0496
Annualized Historical VaR (5.00%),-0.3045,-0.3723,-0.3101,-0.3431,-0.3399,-0.3578
Historical CVaR (5.00%),-0.0626,-0.0663,-0.0674,-0.0633,-0.0550,-0.0659
Annualized Historical CVaR (5.00%),-0.4512,-0.4780,-0.4862,-0.4562,-0.3966,-0.4749
